# An Example of the Pipeline Functionality:

Within this package, we build on the sklearn pipelines to improve their flexibility and use-cases. The main contribution is the modification of sklearn pipelines to fit, transform and predict on data dictionaries.

In [1]:
import sku

import numpy as np
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn import set_config
set_config(display="diagram")

We will download and use a toy-data set available in sklearn:

In [2]:
data = load_digits()

X = data['data']
y = data['target']

Let's make 75% of the data unlabelled, to emphasise the flexibility of this new pipeline. Here, we will class an unlabelled point as having a y value of $-1$.

In [3]:
idx_unlabelled = np.random.choice(y.shape[0], size=int(y.shape[0]*0.75), replace=False)

y[idx_unlabelled] = -1

In [4]:
print(f'There are {y[y!=-1].shape[0]} labelled points '\
        f'and {y[y==-1].shape[0]} unlabelled points.')

There are 450 labelled points and 1347 unlabelled points.


We now make a dictionary containing the data:

In [5]:
train_data = {'X':X[y!=-1], 'y': y[y!=-1], 'X_unlabelled': X[y==-1]}

In [6]:
train_data

{'X': array([[ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  1.,  0.],
        [ 0.,  0.,  5., ..., 13., 12.,  4.],
        ...,
        [ 0.,  0.,  4., ...,  9.,  0.,  0.],
        [ 0.,  0.,  6., ...,  6.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'y': array([1, 1, 2, 2, 7, 0, 5, 9, 5, 2, 7, 2, 2, 4, 9, 9, 5, 8, 0, 1, 2, 3,
        7, 2, 7, 9, 8, 8, 8, 3, 4, 5, 6, 8, 0, 7, 9, 1, 8, 5, 9, 1, 1, 6,
        4, 1, 2, 0, 1, 3, 7, 1, 9, 7, 0, 3, 7, 5, 8, 8, 3, 9, 4, 7, 9, 1,
        3, 5, 7, 9, 0, 9, 3, 5, 3, 6, 6, 1, 9, 8, 2, 6, 4, 1, 7, 3, 0, 1,
        4, 7, 2, 2, 7, 0, 3, 2, 4, 6, 8, 1, 6, 5, 6, 9, 4, 7, 3, 1, 0, 7,
        3, 6, 9, 0, 5, 8, 2, 0, 3, 3, 3, 9, 7, 1, 6, 9, 1, 7, 5, 2, 2, 8,
        9, 8, 8, 2, 3, 5, 0, 4, 5, 6, 7, 8, 0, 5, 5, 4, 7, 7, 0, 2, 7, 6,
        3, 7, 3, 6, 6, 6, 2, 0, 7, 2, 1, 6, 8, 6, 7, 5, 4, 7, 7, 4, 4, 9,
        0, 2, 5, 3, 8, 0, 2, 3, 8, 0, 9, 5, 6, 8, 5, 0, 7, 8, 2, 3, 3, 6,
        4, 2, 7, 2, 1, 6, 1, 3, 6, 3, 1

Pipeline time!

We first define the possible transformers and predictors that we will be using, and wrap them using either the transformer or model wrapper.

In [7]:
pipeline_objects = {

    # fitting the standard scaler on the unlabelled data
    # and transforming the labelled and unlabelled data
    'standard_scaler': sku.SKTransformerWrapperDD(
                            StandardScaler,
                            fit_on=['X_unlabelled'],
                            transform_on=[['X'], ['X_unlabelled']],
                            # any keyword arguments passed here
                            # will be given to the StandardScaler object
                            ),

    # fitting the gbt on the labelled data only
    'gbt': sku.SKModelWrapperDD(
                            HistGradientBoostingClassifier,
                            fit_on=['X', 'y'],
                            predict_on=['X'],
                            # any keyword arguments passed here
                            # will be given to the 
                            # HistGradientBoostingClassifier object
                            )

    }

The wrappers above allow you to wrap any sklearn transformer or model in a way that allows it to accept a data dictionary in its `.fit()` method.

Now we can define a pipeline:

In [8]:
pipeline_name = 'standard_scaler--gbt'

And use the `pipeline_constructor` to build our pipeline:

In [9]:
pipeline = sku.pipeline_constructor(pipeline_name, pipeline_objects)

This looks like:

In [10]:
pipeline

PipelineDD(steps=[['standard_scaler',
                   SKTransformerWrapperDD(copy=True, fit_on=['X_unlabelled'],
                                          transform_on=[['X'],
                                                        ['X_unlabelled']],
                                          transformer=StandardScaler(),
                                          with_mean=True, with_std=True)],
                  ['gbt',
                   SKModelWrapperDD(categorical_features=None,
                                    early_stopping='auto', fit_on=['X', 'y'],
                                    l2_regularization=0.0, learning_rate=0.1,
                                    loss='auto', max_bins=255, max_depth=None,
                                    max_iter=100, max_leaf_nodes=31,
                                    min_samples_leaf=20,
                                    model=HistGradientBoostingClassifier(),
                                    monotonic_cst=None, n_iter_no_change=10,
                                    predict_on=['X'], random_state=None,
                                    scoring='loss', tol=1e-07,
                                    validation_fraction=0.1, verbose=0,
                                    warm_start=False)]])

It is now as easy as sklearn to fit and predict using this pipeline:

In [11]:
pipeline.fit(X = train_data)

PipelineDD(steps=[('standard_scaler',
                   SKTransformerWrapperDD(copy=True, fit_on=['X_unlabelled'],
                                          transform_on=[['X'],
                                                        ['X_unlabelled']],
                                          transformer=StandardScaler(),
                                          with_mean=True, with_std=True)),
                  ['gbt',
                   SKModelWrapperDD(categorical_features=None,
                                    early_stopping='auto', fit_on=['X', 'y'],
                                    l2_regularization=0.0, learning_rate=0.1,
                                    loss='auto', max_bins=255, max_depth=None,
                                    max_iter=100, max_leaf_nodes=31,
                                    min_samples_leaf=20,
                                    model=HistGradientBoostingClassifier(),
                                    monotonic_cst=None, n_iter_no_change=10,
                                    predict_on=['X'], random_state=None,
                                    scoring='loss', tol=1e-07,
                                    validation_fraction=0.1, verbose=0,
                                    warm_start=False)]])

In [12]:
pipeline.predict(X = train_data)

array([1, 1, 2, 2, 7, 0, 5, 9, 5, 2, 7, 2, 2, 4, 9, 9, 5, 8, 0, 1, 2, 3,
       7, 2, 7, 9, 8, 8, 8, 3, 4, 5, 6, 8, 0, 7, 9, 1, 8, 5, 9, 1, 1, 6,
       4, 1, 2, 0, 1, 3, 7, 1, 9, 7, 0, 3, 7, 5, 8, 8, 3, 9, 4, 7, 9, 1,
       3, 5, 7, 9, 0, 9, 3, 5, 3, 6, 6, 1, 9, 8, 2, 6, 4, 1, 7, 3, 0, 1,
       4, 7, 2, 2, 7, 0, 3, 2, 4, 6, 8, 1, 6, 5, 6, 9, 4, 7, 3, 1, 0, 7,
       3, 6, 9, 0, 5, 8, 2, 0, 3, 3, 3, 9, 7, 1, 6, 9, 1, 7, 5, 2, 2, 8,
       9, 8, 8, 2, 3, 5, 0, 4, 5, 6, 7, 8, 0, 5, 5, 4, 7, 7, 0, 2, 7, 6,
       3, 7, 3, 6, 6, 6, 2, 0, 7, 2, 1, 6, 8, 6, 7, 5, 4, 7, 7, 4, 4, 9,
       0, 2, 5, 3, 8, 0, 2, 3, 8, 0, 9, 5, 6, 8, 5, 0, 7, 8, 2, 3, 3, 6,
       4, 2, 7, 2, 1, 6, 1, 3, 6, 3, 1, 0, 5, 3, 6, 9, 4, 8, 2, 8, 6, 7,
       6, 9, 8, 1, 3, 1, 0, 2, 0, 1, 3, 3, 4, 6, 4, 1, 3, 2, 1, 3, 4, 7,
       8, 7, 8, 8, 1, 4, 9, 3, 5, 8, 9, 1, 5, 8, 9, 7, 7, 2, 0, 2, 3, 6,
       9, 9, 8, 2, 3, 3, 7, 8, 4, 0, 6, 2, 5, 6, 7, 0, 4, 9, 6, 9, 8, 4,
       1, 7, 8, 2, 3, 7, 0, 0, 1, 7, 9, 1, 6, 4, 4,

And you can even score, by passing the key for the targets:

In [13]:
print(f"The accuracy on the training data was: {pipeline.score(train_data, 'y')*100}%")

The accuracy on the training data was: 100.0%


But what about trying multiple pipelines?

In [14]:
# set the pipeline names to test
pipeline_names = [
                    'standard_scaler--gbt',
                    'gbt',
                    ]

These can also be run in parallel by specifying `n_jobs`:

In [15]:
pscv = sku.PipelineBasicSearchCV(
        pipeline_names=pipeline_names,
        name_to_object=pipeline_objects,
        metrics={'accuracy':accuracy_score},
        cv=StratifiedKFold(n_splits=5, shuffle=True),
        split_fit_on=['X','y'],
        split_transform_on=['X','y'],
        verbose=True,
        n_jobs=2
        )

In [16]:
pscv

PipelineBasicSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=True),
                      metrics={'accuracy': <function accuracy_score at 0x14af555e0>},
                      n_jobs=2,
                      name_to_object={'gbt': SKModelWrapperDD(categorical_features=None,
                                                              early_stopping='auto',
                                                              fit_on=['X', 'y'],
                                                              l2_regularization=0.0,
                                                              learning_rate=0.1,
                                                              loss='auto',
                                                              max_bins=255,
                                                              max_depth=None,
                                                              max_iter=100,
                                                              m...
                                                              n_iter_no_change=10,
                                                              predict_on=['X'],
                                                              random_state=None,
                                                              scoring='loss',
                                                              tol=1e-07,
                                                              validation_fraction=0.1,
                                                              verbose=0,
                                                              warm_start=False),
                                      'standard_scaler': SKTransformerWrapperDD(copy=True,
                                                                                fit_on=['X_unlabelled'],
                                                                                transform_on=[['X'],
                                                                                              ['X_unlabelled']],
                                                                                transformer=StandardScaler(),
                                                                                with_mean=True,
                                                                                with_std=True)},
                      pipeline_names=['standard_scaler--gbt', 'gbt'],
                      verbose=True)

We must also tell the pipeline searcher which key it should use for the targets when calculating the metrics.

In [17]:
results = pscv.fit(X=train_data, y='y')

Searching: 100%|▉▉▉▉▉▉▉▉▉▉| 10/10 [00:19<00:00,  1.91s/it, pm_n=gbt]


In [18]:
results

,pipeline,repeat_number,split_number,train_or_test,metric,value,splitter,params,train_id,param_updates
0,standard_scaler--gbt,0,0,test,accuracy,0.988889,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
1,standard_scaler--gbt,0,0,train,accuracy,1.000000,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
2,standard_scaler--gbt,0,1,test,accuracy,0.944444,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
3,standard_scaler--gbt,0,1,train,accuracy,1.000000,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
4,standard_scaler--gbt,0,2,test,accuracy,0.977778,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
5,standard_scaler--gbt,0,2,train,accuracy,1.000000,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
6,standard_scaler--gbt,0,3,test,accuracy,0.933333,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
7,standard_scaler--gbt,0,3,train,accuracy,1.000000,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
8,standard_scaler--gbt,0,4,test,accuracy,0.877778,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
9,standard_scaler--gbt,0,4,train,accuracy,1.000000,StratifiedKFold,"{'memory': None, 'steps': [['standard_scaler',...",071c29f9-ad93-433a-9192-975ee1122dc0,None
